In [1]:
include("../src/Julia.jl")

parseData (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
gpath = "../../graphs/";

In [4]:
graphs = readdir("../../graphs");
graphs = graphs[ceil(Int64,length(graphs)/2)+1:ceil(Int64,length(graphs)*3/4)]

6-element Array{ByteString,1}:
 "pathDisjoint_30_exp20"
 "pathRand_100000_u11"  
 "pathRand_1000_u11"    
 "pathRand_1000_u11_dup"
 "rand_1000_iu10"       
 "rand_1000_iu100"      

In [5]:
for cur in graphs
    path = gpath * cur * "/"
    
    try
        a = readFromFile(path * "graph.mtx");
        x = readFromFile(path * "x.vec");
        
        tree = []
        for ind in 1:5
            tr = readFromFile(path * "tree$(ind).mtx");
            
            u,v,w = findnz(tr)
            for i in 1:length(w)
                w[i] = a[u[i],v[i]]
            end
            tr = sparse(u,v,w)
            
            push!(tree, tr);
        end
        
        la = lap(a);
        x = x - mean(x)
        b = la * x; b = b - mean(b);
        
        numIts = 1000;
        @time for i in 1:5
            @time F = treeSolver(tree[i])
            @time F(b);

            # a run on standard Float64 data types
            @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
            norm(lap(a) * myx64 - b) / norm(b)
            insert!(dbg, 1, "$(numIts) iterations")
            insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
            writeToFile(path * "_log_julia_tree$(i)_default.txt", dbg)

            # now useing higer precision data types
            for hp in [64,128,256,512,1024]
                set_bigfloat_precision(hp)
                laHP = toHighPrecision(la; precision=hp)
                treeHP = toHighPrecision(tree[i]; precision=hp)
                xHP = toHighPrecision(x; precision=hp);
                bHP = toHighPrecision(b; precision=hp);
                FHP = treeSolver(treeHP)

                @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=false);
                insert!(dbg, 1, "$(numIts) iterations")
                insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
                writeToFile(path * "_log_julia_tree$(i)_$(hp).txt", dbg)
            end
        end
            
        println("finished running on ", cur, "\n")
    catch
        println("skipping ", cur, "\n")
    end
end

skipping pathDisjoint_30_exp20

skipping pathRand_100000_u11

skipping pathRand_1000_u11

skipping pathRand_1000_u11_dup

  0.529611 seconds (606.99 k allocations: 25.152 MB, 0.93% gc time)
  0.171718 seconds (110.96 k allocations: 4.243 MB)
  1.386172 seconds (18.23 M allocations: 462.461 MB, 5.46% gc time)
117.919060 seconds (617.11 M allocations: 15.488 GB, 26.20% gc time)
121.567976 seconds (616.89 M allocations: 17.004 GB, 26.30% gc time)
136.411217 seconds (616.89 M allocations: 20.053 GB, 25.38% gc time)
154.788961 seconds (616.89 M allocations: 26.152 GB, 26.77% gc time)
183.955783 seconds (616.89 M allocations: 38.349 GB, 28.95% gc time)
  0.001760 seconds (5.14 k allocations: 586.766 KB)
  0.000811 seconds (17.94 k allocations: 344.031 KB)
  1.049068 seconds (18.02 M allocations: 452.727 MB, 13.59% gc time)
113.643220 seconds (616.89 M allocations: 15.479 GB, 26.92% gc time)
128.494607 seconds (616.89 M allocations: 17.004 GB, 26.50% gc time)
138.627750 seconds (616.89 M allo